<a href="https://colab.research.google.com/github/woo2201/sampleCases/blob/reservior/Attribution_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
cd /content/gdrive/MyDrive/dataset/Lifeplanet/Analytics_45000

/content/gdrive/MyDrive/dataset/Lifeplanet/Analytics_45000


In [ ]:
import os
import glob
import numpy as np
import pandas as pd

In [ ]:
path = "/content/gdrive/MyDrive/dataset/Lifeplanet/Analytics_45000/"
filenames = glob.glob(path + "*.csv")
data = [var.split('.')[0].split('/')[-1] for var in filenames]

# dynamically allocate df names.
for name in zip(filenames, data):
        loc = list(name)[0]
        val = list(name)[1]
        globals()[val] = pd.read_csv(loc, sep='delimiter', header=None, engine = 'python')

In [ ]:
data

['Analytics_01',
 'Analytics_02',
 'Analytics_03',
 'Analytics_04',
 'Analytics_05',
 'Analytics_06',
 'Analytics_07',
 'Analytics_08',
 'Analytics_09',
 'Analytics_10']

In [ ]:
def preprocessing(df):

    df = df[6:]
    df.columns = df.columns.astype(str)
    df.rename(columns = {'0':'vector_1'}, inplace = True)

    return df

In [ ]:
df = pd.concat([preprocessing(Analytics_01),
                preprocessing(Analytics_02),
                preprocessing(Analytics_03),
                preprocessing(Analytics_04),
                preprocessing(Analytics_05),
                preprocessing(Analytics_06),
                preprocessing(Analytics_07),
                preprocessing(Analytics_08),
                preprocessing(Analytics_09),
                preprocessing(Analytics_10),])

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
df = df.reset_index()

In [ ]:
df = df.drop(['index'], axis = 1)

In [ ]:
df = df['vector_1'].str.split(',', expand = True)

In [ ]:
df.columns = df.columns.astype(str)

In [ ]:
df = df.add_prefix('temp_')

In [ ]:
df.fillna('-', inplace = True)

In [ ]:
df['rslt'] = df['temp_1'] + df['temp_2'] + df['temp_3'] + df['temp_4'] + df['temp_5']

In [ ]:
df = pd.concat([df, df['rslt'].str.split('₩', expand = True).add_prefix('temps_')], axis = 1)

In [ ]:
df = df.drop(['temp_1', 'temp_2', 'temp_3', 'temp_4', 'temp_5', 'rslt'], axis = 1)

In [ ]:
import re

def clean_text(inputString):

  text_rmv = re.sub('[-=+,#/\?:^.@*\"※~ㆍ!』‘|\(\)\[\]`\'…》\”\“\’·]', ' ', inputString)
  text_rmv = ' '.join(text_rmv.split())

  return text_rmv

In [ ]:
df['temps_0'] = df['temps_0'].apply(lambda x: clean_text(x))
df['temps_1'] = df['temps_1'].apply(lambda x: clean_text(x))

In [ ]:
df['temp_seq'] = df['temp_0'].str.split('>', expand = False)

In [ ]:
df['temp_seq'] = df['temp_seq'].apply(lambda x : [i.strip() for i in x])

In [ ]:
# df['temp_seq'][1335:5151]

In [ ]:
from itertools import groupby
df['temp_seq'] = df['temp_seq'].apply(lambda x : [k for k, _ in groupby(x)])

In [ ]:
df = df[['temp_seq', 'temps_0', 'temps_1']]

In [ ]:
df

,temp_seq,temps_0,temps_1
0,[(direct)],28733,55590133
1,"[kakaopay, (direct)]",13174,90300
2,"[e-insmarket.or.kr, (direct)]",13073,1964547
3,[(direct)],13037,29134711
4,"[pincrux, (direct)]",10987,22388620
...,...,...,...
45626,"[medcost.knia.or.kr, google, e-insmarket.or.kr...",1,0
45627,[https://www.lifeplanet.co.kr/bridge/bl/HPBL95...,1,0
45628,[https://www.lifeplanet.co.kr/bridge/bl/HPBL95...,1,0
45629,[https://www.lifeplanet.co.kr/bridge/bl/HPBL95...,1,0


In [ ]:
cols = pd.DataFrame([pd.Series(x) for x in df.temp_seq])
cols.columns = ['chnn_{}'.format(x+1) for x in cols.columns]

In [ ]:
df = pd.concat([cols, df[['temps_0','temps_1']] ], axis = 1).fillna('-')

In [ ]:
df

,chnn_1,chnn_2,chnn_3,chnn_4,chnn_5,chnn_6,chnn_7,chnn_8,chnn_9,chnn_10,...,chnn_60,chnn_61,chnn_62,chnn_63,chnn_64,chnn_65,chnn_66,chnn_67,temps_0,temps_1
0,(direct),-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,28733,55590133
1,kakaopay,(direct),-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,13174,90300
2,e-insmarket.or.kr,(direct),-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,13073,1964547
3,(direct),-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,13037,29134711
4,pincrux,(direct),-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,10987,22388620
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45626,medcost.knia.or.kr,google,e-insmarket.or.kr,(direct),-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,1,0
45627,https://www.lifeplanet.co.kr/bridge/bl/HPBL950...,(direct),-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,1,0
45628,https://www.lifeplanet.co.kr/bridge/bl/HPBL950...,(direct),https://www.lifeplanet.co.kr/bridge/bl/HPBL950...,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,1,0
45629,https://www.lifeplanet.co.kr/bridge/bl/HPBL950...,(direct),-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,1,0


In [ ]:
df['chnn_1'].value_counts # 242 unique_values

<bound method IndexOpsMixin.value_counts of 0                 (direct)
1                 kakaopay
2        e-insmarket.or.kr
3                 (direct)
4                  pincrux
               ...        
24995               google
24996               google
24997               google
24998               google
24999               google
Name: chnn_1, Length: 25000, dtype: object>

In [ ]:
df['acc_prc'] = round( df['temps_0'].astype(int) / df['temps_0'].astype(int).sum(), 5)

In [ ]:
df = df[df.chnn_2 != '-']

In [ ]:
df.to_csv("rslt.csv")